# Assignment 3
May 10, 2022

Gurman Sachdeva

1007896314

$$\rule{500 px}{4 px}$$

## Question 1

First, we import the required libraries, and the module containing the iteration function:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from iterator import iterator

Next, we create two parallel meshgrids (treating them as 2d arrays). The 'top' grid 
acts as the complex plane, with each point's real and imaginary parts lying on the 
two axes of the grid. The second grid is used to store whether each point on that
complex plane converges or diverges, and how long it takes to diverge.

In [ ]:
real = np.linspace(-2, 2, 500)
imaginary = np.linspace(-2, 2, 500)

REAL, IMAGINARY = np.meshgrid(real, imaginary, indexing='xy')
DIVERGES, DIVERGENCE = np.meshgrid(real, imaginary, indexing='ij')

Now, the iteration function is called on every point in our complex plane. This updates corresponding values in the copy of the complex plane indicating the convergence/divergence of that point through iteration.

In [ ]:
for i in range(REAL.shape[0]):
    for j in range(IMAGINARY.shape[0]):
        DIVERGES[i, j], DIVERGENCE[i, j] = iterator.iterate(REAL[i, j], IMAGINARY[i, j])

Now we create the first plot, only showing whether each point converges. A legend is used rather than a colour bar since there are only two colours, which was largely inspired by the following post:
https://stackoverflow.com/questions/10490302/how-do-you-create-a-legend-for-a-contour-plot-in-matplotlib

In [ ]:
plt.figure(dpi=600)
temp = plt.contourf(real, imaginary, DIVERGES, levels=2)
proxy = [plt.Rectangle((0,0),1,1,fc = pc.get_facecolor()[0]) 
         for pc in temp.collections]
plt.legend(proxy, ["Converges", "Diverges"])
plt.xlabel("Real Part")
plt.ylabel("Imaginary Part")
plt.show()

Now we create the second plot. Note that the darkest colour indicates '0 iterations were taken to diverge', which here actually means the point converges.

In [ ]:
plt.figure(dpi=600)
plt.contourf(real, imaginary, DIVERGENCE, levels=50)
plt.colorbar(label="# Iterations to divergence (0 indicates convergence)")
plt.xlabel("Real Part")
plt.ylabel("Imaginary Part")
plt.show()

## Question 2

We import one more required module:

In [ ]:
from scipy.integrate import solve_ivp

Now we initialize the constants and initial conditions used in the Lorenz equations:

In [ ]:
# initializing parameters
sigma = 10.
r = 28.
b = 8./3.

W0 = np.array([0., 1., 0.])  # The initial conditions for the first integration

Next we define the function representing the system of differential equations of X, Y, and Z:

In [ ]:
def fn_system(t: float, state: np.array) -> np.array:
    """The system of Lorenz differential equations.
    
    Return the value of X', Y', Z' per the Lorenz equations at time <t>.
    
    Parameters
    ----------
    t : float
        The time at which the system is being evaluated.
    
    state : array
        Stores the current values of the parameters X, Y, Z in the equations.
        
    Returns
    -------
    np.array:
        An array containing the values of X', Y', Z' at the input time.
    """
    X = state[0]
    Y = state[1]
    Z = state[2]
    
    X_prime = -sigma * (X - Y)
    Y_prime = (r * X) - Y - (X * Z)
    Z_prime = (-b * Z) + (X * Y)
    
    return np.array([X_prime, Y_prime, Z_prime])

Now we set our range of times and solve the system in all variables over this range:

In [ ]:
times = np.linspace(0, 60, 6000)
result = solve_ivp(fn_system, np.array([0, 60]), W0, t_eval=times)
t = result.t / 0.01
X = result.y[0]
Y = result.y[1]
Z = result.y[2]

Shown below is the plot corresponding to Figure 1 in Lorenz' paper, the evolution of Y with time:

In [ ]:
plt.figure(figsize=(15, 3))
plt.plot(t, Y)
plt.xlabel("Timestep (dimensionless) t/dt")
plt.ylabel("Y")
plt.title("Evolution of Y with time")
plt.show()

We then render the plot corresponding to figure 2 in Lorenz' paper, with plots of Z vs Y and X vs Y:

In [ ]:
plt.figure()
plt.plot(Y, Z)
plt.xlabel("Y")
plt.ylabel("Z")
plt.title("Z w.r.t. Y")
plt.show()

plt.figure()
plt.plot(Y, X)
plt.xlabel("Y")
plt.ylabel("X")
plt.title("X w.r.t. Y")
plt.show()

We now find another solution for the same system with very slightly different initial conditions, W', represented as `W_new`:

In [ ]:
W_new = np.array([0., 1.00000001, 0.])
result_new = solve_ivp(fn_system, np.array([0, 60]), W_new, t_eval=times)
X_new = result_new.y[0]
Y_new = result_new.y[1]
Z_new = result_new.y[2]

Now we can compute the difference between `X` and `X_new`, `Y` and `Y_new`, and `Z` and `Z_new` (simply by subtracting their arrays since both are ordered by time). We take absolute values since we require a measure of distance between `W` and `W_new` w.r.t time.

In [ ]:
X_diff = abs(X - X_new)
Y_diff = abs(Y - Y_new)
Z_diff = abs(Z - Z_new)
W_diff = X_diff + Y_diff + Z_diff

We can now plot the difference between W and W' w.r.t. time:

In [ ]:
plt.figure(figsize=(15, 3))
plt.plot(t, W_diff)
plt.xlabel("Timestep (dimensionless) t/dt")
plt.ylabel("Difference between W and W'")
plt.yscale("log")
plt.title("Separation between states of two systems with different initial conditions, w.r.t time")
plt.show()

We observe that the graph contains three distinguishable regions with different growth characteristics, all of which appear roughly linear; since the y-axis is logarithmic, this indicates that the two states diverge exponentially. We also observe that the growth seems to stagnate in the third region, at around the 3700th timestep. This is likely due to the two states already being the furthest (or nearly the furthest) possible distance from each other in phase space.